In [18]:
import os
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyts.image import GramianAngularField
from PIL import Image

In [2]:
dataPath = "/Users/jwh/Documents/KOTC/수중도킹/2024년도/6_자기장센서 시험/Test0730/Test2/result/"
fileNames = [file for file in sorted(glob.glob(dataPath + "*.csv")) if os.path.isfile(file)]

In [3]:
sensorNames = [f"s{i}" for i in np.arange(1, 7, 1 )]

fileIdx = [1, 8, 9, 10, 11, 12, 13, 14, 15, 2, 3, 4, 5, 6, 7]

numRows = 1920

In [4]:
fileList = [fileNames[idx] for idx in fileIdx]
files = [pd.read_csv(file, sep= ",", nrows= numRows) for file in fileList]

In [5]:
base = pd.read_csv(fileNames[0], sep= ",", nrows= numRows)
meanBase = pd.DataFrame(base.mean(axis= 0), columns= ['base'])

meanBase.head()

,base
s1,-825.509896
s2,-862.213021
s3,-915.934375
s4,820.602083
s5,822.820312


In [6]:
def calibration(df):
    res = [df.iloc[:, i] - meanBase.iloc[i, 0] for i in range(6)]
    res = np.array(res).reshape(6, -1).T
    res = pd.DataFrame(res, columns= sensorNames)
    return res

In [7]:
calibrated = [calibration(files[i]) for i in range(len(files))]
calibrated = np.array(calibrated)
calibrated = calibrated.reshape(15, 6, 1920, 1)
calibrated.shape

(15, 6, 1920, 1)

In [8]:
splitedCase = []
for i in range(calibrated.shape[0]):
    splitedSensor = []
    for j in range(calibrated.shape[1]):
        splitedData = []
        for k in range(int(len(calibrated[i, j]) / 16)):
            splited = calibrated[i, j, k * 16 : (16 + (k * 16)), :]
            splitedData.append(splited)
        splitedData = np.array(splitedData)
        splitedSensor.append(splitedData)
    splitedSensor = np.array(splitedSensor)
    splitedCase.append(splitedSensor)
splitedCase = np.array(splitedCase)

splitedCase.shape

(15, 6, 120, 16, 1)

In [9]:
gaf = GramianAngularField(method= 'difference')

encodedResult = []
for i in range(splitedCase.shape[0]):
    encodedSensor = []
    for j in range(splitedCase.shape[1]):
        encodedData = []
        for k in range(splitedCase.shape[2]):
            data = splitedCase[i, j, k, :]
            encoded = gaf.fit_transform(data.T)
            encoded = encoded.reshape(16, 16)
            encodedData.append(encoded)
        encodedData = np.array(encodedData)
        encodedSensor.append(encodedData)
    encodedSensor = np.array(encodedSensor)
    encodedResult.append(encodedSensor)
encodedResult = np.array(encodedResult)

print(encodedResult.shape)

(15, 6, 120, 16, 16)


In [10]:
concatenateResult = []
for c in range(encodedResult.shape[0]):
    sensor1 = encodedResult[c, 0]
    sensor2 = encodedResult[c, 1]
    sensor3 = encodedResult[c, 2]
    sensor4 = encodedResult[c, 3]
    sensor5 = encodedResult[c, 4]
    sensor6 = encodedResult[c, 5]

    concatenateImg = []
    for d in range(encodedResult.shape[2]):
        sensor1Img = sensor1[d].reshape(16, 16, 1)
        sensor2Img = sensor2[d].reshape(16, 16, 1)
        sensor3Img = sensor3[d].reshape(16, 16, 1)
        sensor4Img = sensor4[d].reshape(16, 16, 1)
        sensor5Img = sensor5[d].reshape(16, 16, 1)
        sensor6Img = sensor6[d].reshape(16, 16, 1)

        res = np.concatenate((sensor1Img, sensor2Img), axis= 2)
        res = np.concatenate((res, sensor3Img), axis= 2)
        res = np.concatenate((res, sensor4Img), axis= 2)
        res = np.concatenate((res, sensor5Img), axis= 2)
        res = np.concatenate((res, sensor6Img), axis= 2)

        concatenateImg.append(res)

    concatenateImg = np.array(concatenateImg)
    concatenateResult.append(concatenateImg)
concatenateResult = np.array(concatenateResult)

print(concatenateResult.shape)

(15, 120, 16, 16, 6)


In [11]:
inIdx = [0, 1, 10]
outIdx = [3, 4, 14]
leftIdx = [2, 7, 8]
rightIdx = [5, 11, 12]

concateIn = [concatenateResult[idx] for idx in inIdx]
concateOut = [concatenateResult[idx] for idx in outIdx]
concateLeft = [concatenateResult[idx] for idx in leftIdx]
concateRight = [concatenateResult[idx] for idx in rightIdx]

concateIn = np.array(concateIn)
concateOut = np.array(concateOut)
concateLeft = np.array(concateLeft)
concateRight = np.array(concateRight)

print("In: ", concateIn.shape)
print("Out: ", concateOut.shape)
print("Left: ", concateLeft.shape)
print("Right: ", concateRight.shape)

In:  (3, 120, 16, 16, 6)
Out:  (3, 120, 16, 16, 6)
Left:  (3, 120, 16, 16, 6)
Right:  (3, 120, 16, 16, 6)


In [14]:
# Image save X

inCounter = 0
outCounter = 0
leftCounter = 0
rightCounter = 0

inClassImg = []
outClassImg = []
leftClassImg = []
rightClassImg = []

inLabelList = []
outLabelList = []
leftLabelList = []
rightLabelList = []

y_in = []
y_out = []
y_left = []
y_right = []

for n in range(3):
    for d in range(concateIn.shape[1]):
        inImg1 = np.clip(concateIn[n, d, :, :, :3], 0., 1.)
        inImg2 = np.clip(concateIn[n, d, :, :, 3:6], 0., 1.)
        outImg1 = np.clip(concateOut[n, d, :, :, :3], 0., 1.)
        outImg2 = np.clip(concateOut[n, d, :, :, 3:6], 0., 1.)
        leftImg1 = np.clip(concateLeft[n, d, :, :, :3], 0., 1.)
        leftImg2 = np.clip(concateLeft[n, d, :, :, 3:6], 0., 1.)
        rightImg1 = np.clip(concateRight[n, d, :, :, :3], 0., 1.)
        rightImg2 = np.clip(concateRight[n, d, :, :, 3:6], 0., 1.)

        inClassImg.append(inImg1)
        inLabelList.append(f"I_{inCounter}")
        y_in.append(0)
        inCounter += 1

        outClassImg.append(outImg1)
        outLabelList.append(f"O_{outCounter}")
        y_out.append(1)
        outCounter += 1
        
        leftClassImg.append(leftImg1)
        leftLabelList.append(f"L_{leftCounter}")
        y_left.append(2)
        leftCounter += 1

        rightClassImg.append(rightImg1)
        rightLabelList.append(f"R_{rightCounter}")
        y_right.append(3)
        rightCounter += 1

        inClassImg.append(inImg2)
        inLabelList.append(f"I_{inCounter}")
        y_in.append(0)
        inCounter += 1

        outClassImg.append(outImg2)
        outLabelList.append(f"O_{outCounter}")
        y_out.append(1)
        outCounter += 1
        
        leftClassImg.append(leftImg2)
        leftLabelList.append(f"L_{leftCounter}")
        y_left.append(2)
        leftCounter += 1

        rightClassImg.append(rightImg2)
        rightLabelList.append(f"R_{rightCounter}")
        y_right.append(3)
        rightCounter += 1

In [12]:
# Image save

"""
basePath = "./Images/"
inPath = os.path.join(basePath, "In/")
outPath = os.path.join(basePath, "Out/)
leftPath = os.path.join(basePath, "Left/")
rightPath = os.path.join(basePath, "Right/")

os.makedirs(inPath, exist_ok= True)
os.makedirs(outPath, exist_ok= True)
os.makedirs(leftPath, exist_ok= True)
os.makedirs(rightPath, exist_ok= True)

counter = 0
for n in range(concateIn.shape[0]):
    for d in range(concateIn.shape[1]):
        img1 = concateIn[n, d, :, :, :3]
        img2 = concateIn[n, d, :, :, 3:6]

        img1 = np.clip(img1, 0., 1.)
        img2 = np.clip(img2, 0., 1.)

        saveInName1 = f"In_{counter}.png"
        filePath1 = os.path.join(inPath, saveInName1)
        plt.imsave(filePath1, img1)

        counter += 1

        saveInName2 = f"In_{counter}.png"
        filePath2 = os.path.join(inPath, saveInName2)
        plt.imsave(filePath2, img2)

        counter += 1

        
counter = 0
for n in range(concateOut.shape[0]):
    for d in range(concateOut.shape[1]):
        img1 = concateOut[n, d, :, :, :3]
        img2 = concateOut[n, d, :, :, 3:6]

        img1 = np.clip(img1, 0., 1.)
        img2 = np.clip(img2, 0., 1.)

        saveOutName1 = f"Out_{counter}.png"
        filePath1 = os.path.join(outPath, saveOutName1)
        plt.imsave(filePath1, img1)

        counter += 1

        saveOutName2 = f"Out_{counter}.png"
        filePath2 = os.path.join(outPath, saveOutName2)
        plt.imsave(filePath2, img2)

        counter += 1


counter = 0
for n in range(concateLeft.shape[0]):
    for d in range(concateLeft.shape[1]):
        img1 = concateLeft[n, d, :, :, :3]
        img2 = concateLeft[n, d, :, :, 3:6]

        img1 = np.clip(img1, 0., 1.)
        img2 = np.clip(img2, 0., 1.)

        saveInName1 = f"Left_{counter}.png"
        filePath1 = os.path.join(leftPath, saveInName1)
        plt.imsave(filePath1, img1)

        counter += 1

        saveInName2 = f"Left_{counter}.png"
        filePath2 = os.path.join(leftPath, saveInName2)
        plt.imsave(filePath2, img2)

        counter += 1

counter = 0
for n in range(concateRight.shape[0]):
    for d in range(concateRight.shape[1]):
        img1 = concateRight[n, d, :, :, :3]
        img2 = concateRight[n, d, :, :, 3:6]

        img1 = np.clip(img1, 0., 1.)
        img2 = np.clip(img2, 0., 1.)

        saveInName1 = f"Right_{counter}.png"
        filePath1 = os.path.join(rightPath, saveInName1)
        plt.imsave(filePath1, img1)

        counter += 1

        saveInName2 = f"Right_{counter}.png"
        filePath2 = os.path.join(rightPath, saveInName2)
        plt.imsave(filePath2, img2)

        counter += 1"""

'\nbasePath = "./Images/"\ninPath = os.path.join(basePath, "In/")\noutPath = os.path.join(basePath, "Out/)\nleftPath = os.path.join(basePath, "Left/")\nrightPath = os.path.join(basePath, "Right/")\n\nos.makedirs(inPath, exist_ok= True)\nos.makedirs(outPath, exist_ok= True)\nos.makedirs(leftPath, exist_ok= True)\nos.makedirs(rightPath, exist_ok= True)\n\ncounter = 0\nfor n in range(concateIn.shape[0]):\n    for d in range(concateIn.shape[1]):\n        img1 = concateIn[n, d, :, :, :3]\n        img2 = concateIn[n, d, :, :, 3:6]\n\n        img1 = np.clip(img1, 0., 1.)\n        img2 = np.clip(img2, 0., 1.)\n\n        saveInName1 = f"In_{counter}.png"\n        filePath1 = os.path.join(inPath, saveInName1)\n        plt.imsave(filePath1, img1)\n\n        counter += 1\n\n        saveInName2 = f"In_{counter}.png"\n        filePath2 = os.path.join(inPath, saveInName2)\n        plt.imsave(filePath2, img2)\n\n        counter += 1\n\n        \ncounter = 0\nfor n in range(concateOut.shape[0]):\n    f

In [15]:
# X data

inClassImg = np.array(inClassImg)
outClassImg = np.array(outClassImg)
leftClassImg = np.array(leftClassImg)
rightClassImg = np.array(rightClassImg)

print(inClassImg.shape)
print(outClassImg.shape)
print(leftClassImg.shape)
print(rightClassImg.shape)

(720, 16, 16, 3)
(720, 16, 16, 3)
(720, 16, 16, 3)
(720, 16, 16, 3)


In [16]:
# y data

y_in = np.array(y_in)
y_left = np.array(y_left)
y_right = np.array(y_right)

print(y_in.shape)
print(y_left.shape)
print(y_right.shape)

(720,)
(720,)
(720,)


In [19]:
# i data

inLabelIdx = np.array(inLabelList)
outLabelIdx = np.array(outLabelList)
leftLabelIdx = np.array(leftLabelList)
rightLabelIdx = np.array(rightLabelList)

print(inLabelIdx.shape)
print(leftLabelIdx.shape)
print(rightLabelIdx.shape)

(720,)
(720,)
(720,)


In [20]:
# Save dataset for MobileNetV2 Training

magnetic = np.concatenate([inClassImg, outClassImg, leftClassImg, rightClassImg])
labelData = np.concatenate([y_in, y_out, y_left, y_right])
indexData = np.concatenate([inLabelIdx, outLabelIdx, leftLabelIdx, rightLabelIdx])

tmp = [[x, y, z] for x, y, z in zip(magnetic, labelData, indexData)]

random.shuffle(tmp)

X_data = [n[0] for n in tmp]
y_data = [n[1] for n in tmp]
i_data = [n[2] for n in tmp]

npzPath = "./"
np.savez(os.path.join(npzPath, "GADF_4Class_16Sample"), X_data= X_data, y_data= y_data, i_data= i_data)